In [96]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.regression import RandomForestRegressor
from pyspark.mllib.evaluation import BinaryClassificationMetrics, RegressionMetrics
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorIndexer, RFormula
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
import matplotlib.pyplot as plt

import numpy as np
import datetime
import atexit




In [97]:
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
import numpy as np
import pandas as pd
from keras.optimizers import sgd
from keras.datasets import cifar10
from keras.utils import np_utils as utils
from keras.layers import Dropout, Dense, Flatten 

In [98]:
#spark = SparkSession.builder.getOrCreate()

#### Add your first Convolution layer
* We first specify that the output will have 32 feature maps 
* We next specifiy a 3x3 Kernel / map
* We set padding to same which means it will not dorp off the edges of the images (look at Stanfort cheat sheet on teams)
* The input size is 32x32 and has 3 channels (RGB)
* Activation is Relu 


In [99]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(32, 32, 3), padding='same', activation='relu'))

*  here we jsut set the input size, kernel size , padding and activation. You can also set the stride

In [100]:
model.add(Conv2D(32,(3,3),padding='valid', activation= 'relu'))

### Pooling
Normally pooling is used for:
* Dimensionality reduction 
* dectecting edges, eyes, noses, corners etc. 
* FUNCTION = reduce the number of parameters and spatial size in the network 

#### Done in 2 ways:
* Max pooling :  States maximum output in rectangular neighbourhood 
* Average pooling: Average output in rectangular neighbourhood 
* reduces amount of pixels, makes image smaller 

In [101]:
model.add(MaxPooling2D(pool_size=(2,2)))

#### Stride
* Modifies movement of videos and images 
* works in conjunction with padding 
* Stride 1 = will move one pixel or unit at a time 
* Stride 2 = will move two pixles or units at a time 
* dilutes how many steops can skipped while scanning features horizontally and vertically on the image
* padding + stride impacts data size, Padding is essential in stride - without padding the next layer will reduce the data size 

##### Works like this:
* starts with the filter in the top left corner and calcuales the value of the first node, t

### Fully Connected Layer
* Summation of all the input and weights which acn determine the final predication 
* output of last pooling layer 
* connectes each node in all the layers (every neuron in one layer to neuros in other layers)
* Performs classification based ont he features extracted by the previous layers 

#### THUS CNN's have to categories : Feature Extraction and Classification based on the extracted features. 

### Non Linear Layers
*  Activiation function applied in last layer ( as you know from ANN) because it gives the final number we use to classify
*  Multiclass uses Softmax
*  RELU + TRIGGER

https://www.cs.toronto.edu/~kriz/cifar.html

In [102]:
(X, y), (X_test, y_test) = cifar10.load_data()

In [103]:
X_test

array([[[[158, 112,  49],
         [159, 111,  47],
         [165, 116,  51],
         ...,
         [137,  95,  36],
         [126,  91,  36],
         [116,  85,  33]],

        [[152, 112,  51],
         [151, 110,  40],
         [159, 114,  45],
         ...,
         [136,  95,  31],
         [125,  91,  32],
         [119,  88,  34]],

        [[151, 110,  47],
         [151, 109,  33],
         [158, 111,  36],
         ...,
         [139,  98,  34],
         [130,  95,  34],
         [120,  89,  33]],

        ...,

        [[ 68, 124, 177],
         [ 42, 100, 148],
         [ 31,  88, 137],
         ...,
         [ 38,  97, 146],
         [ 13,  64, 108],
         [ 40,  85, 127]],

        [[ 61, 116, 168],
         [ 49, 102, 148],
         [ 35,  85, 132],
         ...,
         [ 26,  82, 130],
         [ 29,  82, 126],
         [ 20,  64, 107]],

        [[ 54, 107, 160],
         [ 56, 105, 149],
         [ 45,  89, 132],
         ...,
         [ 24,  77, 124],
        

Why are we deviding by 255 : https://machinelearningmastery.com/how-to-manually-scale-image-pixel-data-for-deep-learning/
 

In [104]:
X , X_test = X.astype('float32')/255.0, X_test.astype('float32')/255.0

Convert our labels to categorical data 

In [105]:
y_test = utils.to_categorical(y_test, 10)

In [106]:
y =utils.to_categorical(y, 10)

In [107]:
Y = utils.to_categorical(Y)

In [108]:
y_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]], dtype=float32)

Initialise sequential mode 

In [109]:
model = Sequential()

In [110]:
model.add(Conv2D(32, (3, 3), input_shape=(32, 32, 3), padding='same', activation='relu'))

Add Drop out rate (Regularization)
https://machinelearningmastery.com/dropout-for-regularizing-deep-neural-networks/#:~:text=A%20good%20value%20for%20dropout,rate%2C%20such%20as%20of%200.8.

In [111]:
model.add(Dropout(0.2))

Another layer with padding set to valid : https://stackoverflow.com/questions/37674306/what-is-the-difference-between-same-and-valid-padding-in-tf-nn-max-pool-of-t#:~:text=To%20sum%20up%2C%20'valid',same'%20padding%20means%20using%20padding.

In [112]:
model.add(Conv2D(32, (3, 3), activation='relu', padding='valid'))

Add a max pooling layer 

In [113]:
model.add(MaxPooling2D(pool_size=(2, 2)))

Flatten the data so that we can classify :https://www.superdatascience.com/blogs/convolutional-neural-networks-cnn-step-3-flattening
* our data is currently in a matrix 
* we need it in a stack 
* this makes the final pooled layer a stack 

In [114]:
model.add(Flatten())

Add the dense layer : which will classify for us, 

In [115]:
model.add(Dense(512, activation='relu'))

Add more dropout to this layer 

In [116]:
model.add(Dropout(0.3))

In [117]:
model.add(Dense(10, activation='softmax'))

Compile the model 

In [118]:
model.compile(loss='categorical_crossentropy', optimizer=sgd(momentum=0.5, decay=0.0004), metrics=['accuracy'])

Fit the algorithm with 25 Epochs

In [119]:
model.fit(X, y, validation_data=(X_test, y_test), epochs=25,  batch_size=512)

Train on 50000 samples, validate on 10000 samples
Epoch 1/25
50000/50000 [==============================] - 41s 821us/step - loss: 2.2145 - accuracy: 0.1763 - val_loss: 2.0726 - val_accuracy: 0.2907
Epoch 2/25
50000/50000 [==============================] - 39s 778us/step - loss: 1.9744 - accuracy: 0.2961 - val_loss: 1.9009 - val_accuracy: 0.3427
Epoch 3/25
50000/50000 [==============================] - 39s 781us/step - loss: 1.8739 - accuracy: 0.3404 - val_loss: 1.8503 - val_accuracy: 0.3575
Epoch 4/25
50000/50000 [==============================] - 39s 788us/step - loss: 1.8050 - accuracy: 0.3649 - val_loss: 1.7626 - val_accuracy: 0.3880
Epoch 5/25
50000/50000 [==============================] - 39s 782us/step - loss: 1.7539 - accuracy: 0.3808 - val_loss: 1.7130 - val_accuracy: 0.4021
Epoch 6/25
50000/50000 [==============================] - 39s 785us/step - loss: 1.7065 - accuracy: 0.4009 - val_loss: 1.6913 - val_accuracy: 0.4152
Epoch 7/25
50000/50000 [==============================] 